In [2]:
a = [1,2,3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


In [4]:
class MyClass:
    def __init__(self):
        self._metrics = {
            'value0': 0,
            'value1': 0
        }
    
    @property
    def get_value0(self):
        return self._metrics['value0']
    

my_class = MyClass()
print(my_class.get_value0)

0


In [7]:
a1 = [1,2,3, 4]
b1 = ["a", "b", "c", "d"]
c1 = [10, 11, 12, 13]
list(zip(a1, b1, c1))

[(1, 'a', 10), (2, 'b', 11), (3, 'c', 12), (4, 'd', 13)]

In [20]:
import numpy as np

np.random.seed(0)

s = np.cumsum(np.round(np.random.poisson(1.9, 180)))
s = s[s < 180]
s

array([  3,   5,   9,  11,  11,  11,  18,  19,  22,  25,  30,  32,  34,
        34,  35,  38,  39,  41,  42,  42,  43,  44,  45,  46,  51,  52,
        53,  54,  54,  56,  57,  60,  60,  62,  63,  64,  66,  68,  70,
        74,  78,  83,  84,  86,  88,  90,  93,  95,  98, 101, 102, 103,
       107, 113, 115, 116, 117, 119, 121, 122, 123, 127, 128, 129, 132,
       133, 134, 136, 136, 136, 139, 141, 142, 144, 148, 149, 151, 154,
       155, 156, 158, 160, 161, 164, 164, 167, 169, 172, 173, 173, 176,
       177])

In [21]:
mu0 = np.log(24_400 / np.sqrt(0.75 ** 2 + 1.0))
sigma0 = np.sqrt(np.log(0.75 ** 2 + 1.0))
arr = np.zeros(180)
arr[s] = np.round(np.random.lognormal(mu0, sigma0, len(s)))
arr

array([    0.,     0.,     0., 27999.,     0.,  4338.,     0.,     0.,
           0., 15516.,     0.,  7095.,     0.,     0.,     0.,     0.,
           0.,     0., 10745., 12391.,     0.,     0., 22224.,     0.,
           0., 10228.,     0.,     0.,     0.,     0., 39849.,     0.,
       43897.,     0., 59901., 26871.,     0.,     0., 20754., 20084.,
           0., 48561., 12834., 17904., 10710., 70895., 22290.,     0.,
           0.,     0.,     0., 17708., 16862., 63362., 23376.,     0.,
       35109., 19624.,     0.,     0., 20578.,     0., 33446., 27611.,
       17929.,     0., 11130.,     0., 14021.,     0., 17813.,     0.,
           0.,     0., 37754.,     0.,     0.,     0., 12078.,     0.,
           0.,     0.,     0., 71246., 22616.,     0., 14212.,     0.,
       22894.,     0., 25596.,     0.,     0., 23846.,     0., 30975.,
           0.,     0.,  5464.,     0.,     0., 23567.,  4234., 13505.,
           0.,     0.,     0., 43737.,     0.,     0.,     0.,     0.,
      

In [54]:
np.random.seed(0) 

def round_to(x, base):
    """
    rounds x to the next base
    """
    assert base > 0, "base must be greater than 0"

    return base * np.round(x/base)



def poisson_int(mu, rbase, size=1) -> int:
    """
    returns the poisson value for mu

    rounds it to an integer
    """
    x = round_to(np.random.poisson(mu, size = size), rbase)

    return np.round(x)


def lognorm_int(mu, varcoef, rbase=1, size=1) -> int:
    """
    returns the lognormal value 
    scales the target expected value mu and varcoef = sigma/mu to the 
        embedded normal distribution with mu0 and sigma0 and returns
        the corresponding output
    beauty of lognorm: is always > 0

    rounds it to an integer
    """

    mu0 = np.log(mu / np.sqrt(varcoef ** 2 + 1.0))
    sigma0 = np.sqrt(np.log(varcoef ** 2 + 1.0))
    
    x = round_to(np.random.lognormal(mu0, sigma0, size = size), rbase)

    #return random.lognormvariate(mu0, sigma0)
    return np.round(x)


def gen_demands(d_mu, d_varcoef, d_t_mu, fcast_period):
    # generate a list of poisson values as with mu = d_t_mu and round it to int
    poisson_idx = np.round(np.random.poisson(d_t_mu, fcast_period))
    # calculate the cumulative sum to have it as indices and then filter out the ones with are grater than the forecast period
    poisson_idx = np.cumsum(poisson_idx)
    poisson_idx = poisson_idx[poisson_idx < fcast_period]

    # generate an array of zeros of demands
    demands = np.zeros(fcast_period)
    # fill the demands with the poisson values
    demands[poisson_idx] = lognorm_int(d_mu, d_varcoef, size = len(poisson_idx))

    return demands


gen_demands(24_400, 0.75, 1.9, 180)

array([    0.,     0.,     0., 27999.,     0.,  4338.,     0.,     0.,
           0., 15516.,     0.,  7095.,     0.,     0.,     0.,     0.,
           0.,     0., 10745., 12391.,     0.,     0., 22224.,     0.,
           0., 10228.,     0.,     0.,     0.,     0., 39849.,     0.,
       43897.,     0., 59901., 26871.,     0.,     0., 20754., 20084.,
           0., 48561., 12834., 17904., 10710., 70895., 22290.,     0.,
           0.,     0.,     0., 17708., 16862., 63362., 23376.,     0.,
       35109., 19624.,     0.,     0., 20578.,     0., 33446., 27611.,
       17929.,     0., 11130.,     0., 14021.,     0., 17813.,     0.,
           0.,     0., 37754.,     0.,     0.,     0., 12078.,     0.,
           0.,     0.,     0., 71246., 22616.,     0., 14212.,     0.,
       22894.,     0., 25596.,     0.,     0., 23846.,     0., 30975.,
           0.,     0.,  5464.,     0.,     0., 23567.,  4234., 13505.,
           0.,     0.,     0., 43737.,     0.,     0.,     0.,     0.,
      

In [1]:
QUANT_MU_1 = 24_400
DIST_DAYS_1 = 1.9
p1 = {
    "d_mu"              : QUANT_MU_1,                                           # number of pieces per day produced
    "d_varcoef"         : 0.75,                                                 # variation coefficient for demand determination
    "d_t_mu"            : DIST_DAYS_1,                                          # number of days on average between 2 orders
    "d_fc_noise"        : 1.0,                                                  # variation coefficient for forecast determination 0 to 1 as percentage on top of it
    "safety_stock"      : 1.0 * (QUANT_MU_1 / DIST_DAYS_1) * 7,     # safety stock for one week
    "E_p"               : 0.03,                                                 # cost per piece for EOQ calculation
    "B_k"               : 200,                                                  # setup cost per batch for EOQ calculation  
    "irate"             : 0.1,                                                  # interest rate for inventory for EOQ calculation
    "t_e"               : 0.22 / (60*60*24),                                   # time to produce one piece in fraction of a day      
    "t_r"               : 2.0 / 24,                                  # time to setup a batch in fraction of a day            
    "eoq_mode"          : "FOW"                                                 # mode for EOQ calculation (check below the definitions)
}

p1.d_mu

AttributeError: 'dict' object has no attribute 'd_mu'

In [3]:
a = 1
a += 3-1
a

3

In [1]:
listt = [1,2,3,4,5,6,7,8,9,10]

listt[:0]

import numpy as np

np.mean(listt)

5.5

In [2]:
a = 2
b = 5

dictt = {
    "a": a,
    "b": b
}

a = 10
dictt["a"]

2